## Código final consolidado: LSTM com HPO + linearidade + robustez

In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Flatten, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1_l2
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from kerastuner.tuners import RandomSearch
import joblib
import os

# === Função para criar dataset multi-step ===
def criar_dataset_multi_step(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        X.append(series[i:i+look_back])
        y.append(series[i+look_back+passo-1])
    return np.array(X), np.array(y)

# === Função geradora de modelo com look_back fixado ===
def get_build_model(look_back):
    def build_model(hp):
        inputs = Input(shape=(look_back, 1))
        lstm_out = LSTM(
            units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
            return_sequences=False
        )(inputs)

        flattened = Flatten()(inputs)
        normalized = BatchNormalization()(flattened)
        linear_weights = Dense(
            1, use_bias=False,
            kernel_regularizer=l1_l2(l1=0.01, l2=0.01),
            name="linear_combination"
        )(normalized)

        combined = Concatenate()([lstm_out, linear_weights])
        output = Dense(1)(combined)

        model = Model(inputs, output)
        model.compile(
            optimizer='adam',
            loss=tf.keras.losses.Huber(delta=1.0),
            metrics=['mae']
        )
        return model
    return build_model

# === Loop para cada horizonte de previsão ===
a3_df = pd.read_csv("A3_component.csv")
a3 = a3_df["A3"].values.reshape(-1, 1)

for passo in [1, 5, 7, 30]:
    look_back = 5 if passo <= 5 else 10
    print(f"\n🚀 Treinando LSTM para t+{passo} com look_back={look_back}")

    # Normalização
    scaler = MinMaxScaler()
    a3_scaled = scaler.fit_transform(a3)
    joblib.dump(scaler, f"scaler_A3_t{passo}.joblib")

    # Criar dataset
    X, y = criar_dataset_multi_step(a3_scaled, look_back=look_back, passo=passo)

    # Tuner
    tuner = RandomSearch(
        get_build_model(look_back),
        objective='val_loss',
        max_trials=5,
        executions_per_trial=1,
        directory='tuner_results',
        project_name=f'lstm_t{passo}'
    )

    # Validação cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        tuner.search(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=50,
            batch_size=32,
            callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
            verbose=0
        )
        break  # usar apenas a 1ª divisão do TimeSeriesSplit

    # Treinar melhor modelo
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=32,
        callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
        verbose=1
    )

    # Salvar modelo
    best_model.save(f"lstm_a3_t{passo}.keras")
    print(f"✅ Modelo salvo: lstm_a3_t{passo}.keras")


C:\Users\marco\AppData\Local\Temp\ipykernel_25868\4086292502.py:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch



🚀 Treinando LSTM para t+1 com look_back=5
Reloading Tuner from tuner_results\lstm_t1\tuner0.json

Epoch 1/100


C:\Users\marco\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 6.6009e-04 - mae: 0.0278 - val_loss: 4.5126e-04 - val_mae: 0.0249
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.8754e-04 - mae: 0.0213 - val_loss: 4.1534e-04 - val_mae: 0.0237
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.8140e-04 - mae: 0.0211 - val_loss: 3.7248e-04 - val_mae: 0.0225
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.2018e-04 - mae: 0.0229 - val_loss: 6.9821e-04 - val_mae: 0.0326
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.3398e-04 - mae: 0.0197 - val_loss: 2.9520e-04 - val_mae: 0.0197
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.6993e-04 - mae: 0.0175 - val_loss: 2.8983e-04 - val_mae: 0.0194
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.5010e-04 - mae: 0.0172 - val_loss: 3.2448e-04 - val_mae: 0.0209
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.8964e-04 - mae: 0.0182 - val_loss: 3.2710e-04 - val_mae: 0.0210
Epoch 9/100

C:\Users\marco\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0048 - mae: 0.0757 - val_loss: 0.0054 - val_mae: 0.0847
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0045 - mae: 0.0731 - val_loss: 0.0061 - val_mae: 0.0908
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0042 - mae: 0.0708 - val_loss: 0.0066 - val_mae: 0.0953
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0048 - mae: 0.0760 - val_loss: 0.0071 - val_mae: 0.0996
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0047 - mae: 0.0757 - val_loss: 0.0051 - val_mae: 0.0823
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0042 - mae: 0.0706 - val_loss: 0.0053 - val_mae: 0.0839
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0043 - mae: 0.0712 - val_loss: 0.0053 - val_mae: 0.0847
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0041 - mae: 0.0699 - val_loss: 0.0057 - val_mae: 0.0873
Epoch 9/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0047 - mae: 0.

C:\Users\marco\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0065 - mae: 0.0882 - val_loss: 0.0056 - val_mae: 0.0851
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0058 - mae: 0.0830 - val_loss: 0.0060 - val_mae: 0.0880
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0064 - mae: 0.0872 - val_loss: 0.0065 - val_mae: 0.0938
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0063 - mae: 0.0875 - val_loss: 0.0049 - val_mae: 0.0789
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0064 - mae: 0.0875 - val_loss: 0.0056 - val_mae: 0.0857
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0057 - mae: 0.0813 - val_loss: 0.0044 - val_mae: 0.0737
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0057 - mae: 0.0821 - val_loss: 0.0051 - val_mae: 0.0805
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0060 - mae: 0.0851 - val_loss: 0.0053 - val_mae: 0.0833
Epoch 9/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0055 - mae: 0.

C:\Users\marco\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0091 - mae: 0.1054 - val_loss: 0.0141 - val_mae: 0.1406
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0087 - mae: 0.1029 - val_loss: 0.0155 - val_mae: 0.1487
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0089 - mae: 0.1046 - val_loss: 0.0156 - val_mae: 0.1488
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0086 - mae: 0.1031 - val_loss: 0.0127 - val_mae: 0.1333
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0087 - mae: 0.1024 - val_loss: 0.0136 - val_mae: 0.1378
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0093 - mae: 0.1051 - val_loss: 0.0155 - val_mae: 0.1476
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0088 - mae: 0.1041 - val_loss: 0.0151 - val_mae: 0.1457
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae: 0.0994 - val_loss: 0.0148 - val_mae: 0.1442
Epoch 9/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0093 - mae: 0.